In [28]:
import os
import psycopg2
from psycopg2 import sql

# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"

def insert_values_into_table(values_list):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")
        cur = conn.cursor()

        
        args = ','.join(cur.mogrify("(%s,%s,%s, %s, %s, %s, %s)", i).decode('utf-8')
                for i in values_list)
        
        cur.execute("INSERT INTO hm31.pubmed_all_xmls VALUES " + (args))

        conn.commit()
        print("Inserted values successfully")

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()

# # Example usage:
# values_to_insert = [
#     (123425, '', '', '', 0, 0, ''),
#     (543251, '5678-1234', 'Grant3, Grant4', 'Chemical3, Chemical4', 2021, 2020, 'Term3, Term4')
# ]

# insert_values_into_table(values_to_insert)


Inserted values successfully


In [39]:
import xmltodict

def parse(file_path):
    try:
        # Open and read the XML file
        with open(file_path, 'r', encoding='utf-8') as file:
            xml_contents = file.read()

        # Parse the XML content using xmltodict
        print('red')
        xml_dict = xmltodict.parse(xml_contents)
        
        # Print the parsed XML as a Python dictionary
        print(len(xml_dict))
        
    except Exception as e:
        print("Error:", e)
        
    return xml_dict

# Example usage:
# xml_dict = parse('/shared/hm31/xml_data/pubmed23n0001.xml')



red
1


In [41]:
from tqdm import tqdm

def parse_single_record(xml_dict):
    mesh_headings = []
    grants = []
    year = ""
    journal_ISSN = ""
    chemical_list = []
    meta_data = {}
    pub_year = ""
    PMID = ""

    try:
        xml_dict = dict(xml_dict)
        
        
#         print(xml_dict)
        
        try:
            if 'PMID' in xml_dict:
                PMID = xml_dict['PMID']['#text']
        except:
            pass
        
        try:
            if 'DateCompleted' in xml_dict:
                new_dic = dict(xml_dict['DateCompleted'])
                if 'Year' in new_dic:
                    year = new_dic['Year']

            else:
                  pass
    
        except:
            pass

        
        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])
                journal_ISSN = new_dic['Journal']['ISSN']['#text']
                
        except:
            pass
    
        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])
                pub_year = new_dic['Journal']['JournalIssue']['PubDate']['Year']

        except:
            pass
     
        

        try:
            if 'Article' in xml_dict: 
                new_dic = dict(xml_dict['Article'])
                
            if 'Title' in  new_dic['Journal'].keys():
                journal_title = new_dic['Journal']['Title']
        
        except:
            pass

        try:
            if 'Article' in xml_dict: 
                new_dic = dict(xml_dict['Article'])

            if 'GrantList' in new_dic:
                if type(new_dic['GrantList']['Grant']) == list:
                    for grant in new_dic['GrantList']['Grant']:
                        if 'GrantID' in grant:
                            grants.append((grant['GrantID']))
                            
                else:
                     grants.append(new_dic['GrantList']['Grant']['GrantID'])
                     pass   
        except:
            pass
        
        
        try:
            if 'MeshHeadingList' in xml_dict:  
                if type(xml_dict['MeshHeadingList']['MeshHeading']) == list:
                    for mesh in xml_dict['MeshHeadingList']['MeshHeading']:
                        if '@Type' in mesh['DescriptorName'].keys() and mesh['DescriptorName']['@Type'] == 'Geographic':
                            continue
                
                
                        mesh_headings.append((mesh['DescriptorName']['@UI']))
            
                else:
                    
                    mesh = xml_dict['MeshHeadingList']['MeshHeading']['DescriptorName']
                    if  (not '@Type' in mesh.keys() or mesh['@Type'] != 'Geographic'):
                        mesh_headings.append(xml_dict['MeshHeadingList']['MeshHeading']['DescriptorName']['@UI'])
                    
        except:
            pass
        
        
        try:
        
            if 'ChemicalList' in xml_dict:
               if type(xml_dict['ChemicalList']['Chemical']) == list:
                   for substance in xml_dict['ChemicalList']['Chemical']:
                        if substance['NameOfSubstance']['@UI'][0].lower() == 'c':
                            chemical_list.append(substance['NameOfSubstance']['@UI'])
                        
               else:
                       if substance['NameOfSubstance']['@UI'][0].lower() == 'c':
                           xml_dict['ChemicalList']['Chemical']['NameOfSubstance']['@UI'][0]
                           xxx = input('third')
                            
        except:
            pass
        
    except:
        pass


    if len(year) == 0:
        year = pub_year
        
    meta_data = {'PMID': PMID, 'mesh': mesh_headings, 'grants': grants, 'year': year, 
                 'journal_ISSN': journal_ISSN,'journal_title': journal_title,
                 'chemical' : chemical_list, 'pub_year': pub_year} 
    
    return meta_data

meta_data_array = []

for i in tqdm(range(len(xml_dict['PubmedArticleSet']['PubmedArticle']))):
    rec = xml_dict['PubmedArticleSet']['PubmedArticle'][i]['MedlineCitation']
    x=parse_single_record(rec)
    meta_data_array.append(x)
    
    

100%|██████████| 30000/30000 [00:02<00:00, 12930.16it/s]


In [36]:
#Convert dict to query
#meta_data = {'mesh': mesh_headings, 'grants': grants, 'year': year, 
#                 'journal_ISSN': journal_ISSN,'journal_title': journal_title,
#                 'chemical' : chemical_list, 'pub_year': pub_year} 

#(543251, '5678-1234', 'Grant3, Grant4', 'Chemical3, Chemical4', 2021, 2020, 'Term3, Term4')
def convert_dict_to_query(dic):
    if dic['year'] == '':
        year = 0
    
    else:
        year = dic['year']
        
    
    if dic['pub_year'] == '':
        pub_year = 0
    
    else:
        pub_year = dic['pub_year']
        
        
    query = (int(dic['PMID']), dic['journal_ISSN'], ' '.join(dic['grants']), ' '.join(dic['chemical']), 
            pub_year, year, ' '.join(dic['mesh']) )
    
    return query

print(meta_data_array[0])

{'PMID': '1', 'mesh': ['D000445', 'D000818', 'D001826', 'D002245', 'D005561', 'D000882', 'D006801', 'D006863', 'D007700', 'D000432', 'D008722', 'D011549'], 'grants': ['F32 AG064886', 'MC_UU_12013/5'], 'year': '1976', 'journal_ISSN': '0006-2944', 'journal_title': 'Biochemical medicine', 'chemical': [], 'pub_year': '1975'}


In [38]:
query_array = []

for i in meta_data_array:
    query_array.append(convert_dict_to_query(i))
    
insert_values_into_table(query_array)

Inserted values successfully


In [42]:
def parallelize(file_path):
    xml_dict = parse(file_path)
    
    meta_data_array = []

    for i in range(len(xml_dict['PubmedArticleSet']['PubmedArticle'])):
        rec = xml_dict['PubmedArticleSet']['PubmedArticle'][i]['MedlineCitation']
        x=parse_single_record(rec)
        meta_data_array.append(x)
        
    query_array = []

    for i in meta_data_array:
        query_array.append(convert_dict_to_query(i))

    
    insert_values_into_table(query_array)


In [43]:
import multiprocessing

nest_dir = '/shared/hm31/xml_data/'
files = os.listdir(nest_dir)

lst = []

for file in files:
    lst.append((nest_dir+file,))
    
with multiprocessing.Pool(processes=100) as pool:
    results = pool.starmap(parallelize, lst)


1166
